## Getting Chat Generated Articles from OpenAI

In [ ]:
#%pip install openai

In [ ]:
import openai
import pandas as pd
import time

# Set api key here, left empty for account privacy reasons. You can add you own api key if you want to try
openai.api_key = ""

In [ ]:
# with open("C:\\Users\\graduate\\Box\\Courses taken\\CS 6830 Data Science in practice\\Final project\\openai_api.txt", 'r') as f:
#     api_key = f.read().strip()

# # Set api key here, left empty for account privacy reasons. You can add you own api key if you want to try
# openai.api_key = api_key

In [ ]:
# articles is a list of dictionaries. Each dictionary contains all the information needed to generate one news article.
# Information is gotten by scraping existing articles online
# Currently, the prompt requires the the new organization: 'source', wordcount the generated article should be: 'wordcount'
# and the headline: 'headline'. An example is shown below
articles = [{'source':'CNN', 'wordcount':'200', 'headline':'What is the Good Friday Agreement? How a historic deal ended the Troubles in Northern Ireland'}]

In [ ]:
def getFakeArticles(articles):
    fakeArticles = pd.DataFrame(columns=['Headline', 'Content'])
    for article in articles:
        prompt=[
            {"role": "system", "content": f"You are a journalist at {article['source']}"},
            {"role": "user", "content": f"Write an article with about {article['wordcount']} words with the headline '{article['headline']}'"},
        ]
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=prompt
        )
        fakeArticles.loc[len(fakeArticles.index)] = [article['headline'], response['choices'][0]['message']['content']]
        print(f'Articles created: {len(fakeArticles)}')

        time.sleep(5)

    return fakeArticles

## Make CNN Fake Articles

In [ ]:
# df = pd.read_csv('CNN500_items.csv')

# ai_50 = df.sample(frac = 0.5)
# real_50 = df.drop(ai_50.index)

# articles = []
# for index, row in ai_50.iterrows():
#     articles.append({'source':'CNN','wordcount':len(row.Description.replace('\n','').split(' ')),'headline':row.Title.replace('\n','').strip()})

In [ ]:
# ai_gen = getFakeArticles(articles)

In [ ]:
# ai_gen.columns = ['Title', 'Content']
# ai_gen['Source'] = 1

# non_ai = real_50[['Title', 'Description']]
# non_ai.columns = ['Title', 'Content']
# non_ai['Source'] = 0

# cnn_mixed = pd.concat([ai_gen, non_ai], ignore_index=True)

In [ ]:
# cnn_mixed.to_csv(path_or_buf="cnn_mixed.csv")

## Make AP Fake Articles

In [ ]:
# df = pd.read_csv('AP501dataset.csv')

# df.loc[0].text
# ai_50 = df.sample(frac = 0.5)
# real_50 = df.drop(ai_50.index)

# articles = []
# for index, row in ai_50.iterrows():
#     articles.append({'source':'AP News','wordcount':len(row.text.replace('\n','').split(' ')),'headline':row.title.replace('\n','').strip()})

In [ ]:
# ai_gen = getFakeArticles(articles)

In [ ]:
# ai_gen.columns = ['Title', 'Content']
# ai_gen['Source'] = 1

# non_ai = real_50[['title', 'text']]
# non_ai.columns = ['Title', 'Content']
# non_ai['Source'] = 0

# ap_mixed = pd.concat([ai_gen, non_ai], ignore_index=True)

In [ ]:
# ap_mixed.to_csv("ap_mixed.csv")

## Make Fake Fox News Articles

In [ ]:
# df = pd.read_csv('fox260.csv')

# ai_50 = df.sample(frac = 0.5)
# real_50 = df.drop(ai_50.index)

# articles = []
# for index, row in ai_50.iterrows():
#     articles.append({'source':'Fox News','wordcount':len(row.text.replace('\n','').split(' ')),'headline':row.title.replace('\n','').strip()})

In [ ]:
# ai_gen = getFakeArticles(articles)

In [ ]:
# ai_gen.columns = ['Title', 'Content']
# ai_gen['Source'] = 1

# non_ai = real_50[['title', 'text']]
# non_ai.columns = ['Title', 'Content']
# non_ai['Source'] = 0

# fox_mixed = pd.concat([ai_gen, non_ai], ignore_index=True)

In [ ]:
# fox_mixed.to_csv("fox_mixed.csv")